In [ ]:
from langgraph.graph import StateGraph, END
from pydantic import BaseModel
from typing import List, Optional
import random
from IPython.display import Image, display
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
import os
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import JsonOutputParser


# Estado

In [ ]:
class Estado(BaseModel):
    modo: Optional[str] = None
    input: Optional[str] = None
    nivel: Optional[str] = None
    temas: Optional[List[str]] = []
    tema_actual: Optional[int] = 0
    ultima_pregunta: Optional[str] = None  
    aprobo_repaso: Optional[bool] = None
    fortalezas: Optional[List[str]] = []
    debilidades: Optional[List[str]] = []

# Preguntas Quiz

In [ ]:
quiz_preguntas = {
    "basico": [
        {"tema": "Variables aleatorias", "pregunta": "¿Qué es una variable aleatoria?"},
        {"tema": "Probabilidad", "pregunta": "¿Qué es la probabilidad clásica?"},
        {"tema": "Distribuciones simples", "pregunta": "¿Qué es una distribución uniforme?"},
        {"tema": "Eventos", "pregunta": "¿Qué es un evento en probabilidad?"}
    ],
    "intermedio": [
        {"tema": "Desviación estándar", "pregunta": "¿Qué es la desviación estándar?"},
        {"tema": "Medidas de dispersión", "pregunta": "¿Qué es la varianza?"},
        {"tema": "Estadística descriptiva", "pregunta": "¿Qué es la media aritmética?"},
        {"tema": "Distribuciones", "pregunta": "¿Qué es una distribución normal?"}
    ],
    "avanzado": [
        {"tema": "Probabilidad conjunta", "pregunta": "¿Cómo se calcula la probabilidad conjunta de dos eventos independientes?"},
        {"tema": "Teorema de Bayes", "pregunta": "Explica el teorema de Bayes con un ejemplo."},
        {"tema": "Distribuciones avanzadas", "pregunta": "¿Qué es una distribución binomial?"},
        {"tema": "Inferencia", "pregunta": "¿Qué es una estimación puntual en inferencia estadística?"}
    ]
}

# Temas por nivel

In [ ]:
temas_por_nivel = {
    "basico": ["Conceptos básicos de probabilidad", "Variables aleatorias", "Distribuciones simples"],
    "intermedio": ["Estadística descriptiva", "Medidas de dispersión", "Desviación estándar"],
    "avanzado": ["Probabilidad conjunta", "Teorema de Bayes", "Distribuciones avanzadas"]
}

# Configuracion LLM

In [ ]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(
    model="o4-mini-2025-04-16",
    openai_api_key=openai_api_key
)

# Nodos

## nodo_inicio

In [ ]:
prompt_clasificar = PromptTemplate(
    input_variables=["input"],
    template=(
        "Clasifica el siguiente mensaje del usuario como 'modo libre' o 'modo guiado'. "
        "Solo responde con una de esas dos opciones.\n\n"
        "Mensaje: {input}\n"
        "Clasificación:"
    )
)

In [ ]:
classifier_chain = LLMChain(
    llm=llm,
    prompt=prompt_clasificar
)

In [ ]:
## nodo
def classify_mode(state):
    user_input = state["input"]
    respuesta = classifier_chain.invoke(input=user_input)
    modo = respuesta.get("text", "").strip().lower()
    if "libre" in modo:
        state["modo"] = "modo libre"
    else:
        state["modo"] = "modo guiado"
    return state

In [ ]:
#edge
def transicion_inicio(state: Estado):
    if state.modo == "modo libre":
        return "quiz_nivel"
    else:
        return "libre"

## nodo quiz_nivel

In [ ]:
prompt_quiz = ChatPromptTemplate.from_messages([
    ("system",
     """Eres un experto en educación. Evalúa las siguientes respuestas del usuario a preguntas de probabilidad y estadística.
Para cada respuesta, califica de 0 a 5 (donde 0 es incorrecta y 5 es perfecta), explica brevemente la calificación.
Al final, resume las fortalezas y debilidades del usuario por tema y sugiere el nivel adecuado (básico, intermedio, avanzado) según el promedio de los puntajes:
- Básico: promedio < 2.5
- Intermedio: 2.5 <= promedio < 4
- Avanzado: promedio >= 4

Devuelve la respuesta SOLO en formato JSON con la siguiente estructura:
{{
  "nivel": "basico/intermedio/avanzado",
  "fortalezas": ["tema1", "tema2", ...],
  "debilidades": ["tema1", "tema2", ...],
  "detalle": [
    {{
      "pregunta": "...",
      "respuesta": "...",
      "tema": "...",
      "puntaje": 0-5,
      "feedback": "..."
    }},
    ...
  ]
}}

Respuestas del usuario:
{respuestas_usuario}
""")
])

In [ ]:
parser = JsonOutputParser()
chain_quiz = LLMChain(
    llm=llm,
    prompt=prompt_quiz,
    output_parser=parser
)

In [ ]:
# nodo
def nodo_quiz_nivel(state):
    print("\nVamos a hacer un quiz para conocer tu nivel.")
    respuestas_usuario = []
    for nivel, preguntas in quiz_preguntas.items():
        seleccionadas = random.sample(preguntas, 1)  # 1 aleatoria por nivel
        for q in seleccionadas:
            resp = input(q["pregunta"] + " (responde brevemente): ")
            respuestas_usuario.append({
                "nivel": nivel,
                "tema": q["tema"],
                "pregunta": q["pregunta"],
                "respuesta": resp
            })
            
    result = chain_quiz.invoke({"respuestas_usuario": str(respuestas_usuario)})
    
    data = result["text"]
    state.nivel = data["nivel"]
    state.fortalezas = data["fortalezas"]
    state.debilidades = data["debilidades"]
    state.detalle = data["detalle"]
    
    return state

## nodo plan_estudio

In [ ]:
##